In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import xarray as xr
from pyCIAM.run import execute_pyciam
from shared import (
    AUTHOR,
    BIN_MAP_PATH,
    CONTACT,
    HISTORY,
    PATH_OUTPUT,
    PATH_PARAMS,
    PATH_REFA,
    PATH_SLIIDERS,
    PATH_SLIIDERS_SEG,
    PATH_SLR_INT,
    PATHS_SURGE_LOOKUP,
    SEG_CHUNKSIZE,
    TMPPATH,
    start_dask_cluster,
)

In [3]:
DESCRIPTION = "pyCIAM outputs using impact-region SLIIDERS database from DSCIM-EPA."

In [4]:
from dask_gateway import Gateway

gateway = Gateway()
gateway.list_clusters()

[]

In [5]:
client, cluster = start_dask_cluster(
    env_items={
        "DASK_DISTRIBUTED__SCHEDULER__ALLOWED_FAILURES": "10",
    }
)
cluster.scale(1000)

# cluster = gateway.connect(gateway.list_clusters()[0].name)
# client = cluster.get_client()

# from distributed import Client

# client = Client(n_workers=7)
# cluster = client.cluster

cluster

## Get filter of scenarios to run (comment out if we are not running full set of scenarios)

In [6]:
scen_mc_filter = (
    xr.open_zarr(str(BIN_MAP_PATH), chunks=None)[["scenario", "sample"]]
    .to_dataframe()
    .set_index(["scenario", "sample"])
    .index.unique()
)
scen_mc_filter = scen_mc_filter.union(
    pd.MultiIndex.from_product(
        (["ncc_ar6"], scen_mc_filter.get_level_values("sample").unique()),
        names=["scenario", "sample"],
    )
)

## Run pyCIAM

In [7]:
def postprocess_func(ds):
    return (
        ds.drop_vars("npv").sel(case=["noAdaptation", "optimalfixed"]).sum("costtype")
    )


common_kwargs = dict(
    pyciam_seg_chunksize=SEG_CHUNKSIZE,
    surge_input_paths=PATHS_SURGE_LOOKUP,
    dask_client_func=lambda: client,
    remove_tmpfile=False,
    seg_var="seg_ir",
    adm_var="impact_region",
    mc_dim="sample",
    quantiles=None,
    no_surge_check=True,
    other_chunksizes={"ssp": 1, "iam": 1, "scen_mc": 1000},
    extra_attrs={
        "author": AUTHOR,
        "contact": CONTACT,
        "description": DESCRIPTION,
        "history": HISTORY,
    },
    scen_mc_filter=scen_mc_filter,
    postprocess_func=postprocess_func,
)

### Run normal pyCIAM (full-adapt and no-adapt)

In [10]:
execute_pyciam(
    PATH_PARAMS,
    PATH_SLIIDERS,
    PATH_SLR_INT,
    "ar6",
    PATH_REFA,
    econ_input_path_seg=PATH_SLIIDERS_SEG,
    output_path=PATH_OUTPUT,
    tmp_output_path=TMPPATH,
    **common_kwargs,
)

Creating ESL impacts lookup table if it does not exist...
Baseline adaptation RPs already calculated...
Queueing jobs to calculate costs for all adaptation scenarios...


/srv/conda/envs/notebook/lib/python3.14/site-packages/xarray/structure/chunks.py:180: PerformanceWarning: Increasing number of chunks by factor of 32
  _, chunked_data = chunkmanager.unify_chunks(*unify_chunks_args)


### Run fixed adaptation - THIS SECTION AND ALL FOLLOWING HAS NOT YET BEEN TESTED/RUN AS OF 2/4/26

In this run, we hold rho (resilience to storm impacts) fixed at 2000 levels (the only year where values are the same across SSPs and IAMs). We do **NOT** regenerate "refA", which defines present-day adaptation levels by running pyCIAM under a no-climate-change scenario and picking the optimal adaptation height. In other words, we are using the time-varying rho to define refA but then fixing rho at 2000 levels for the actual model run.

In [ ]:
tmp = xr.open_zarr(str(PATH_SLIIDERS))
tmp["rho"] = tmp.rho.sel(year=2000).expand_dims(year=tmp.year)
tmp.to_zarr(str(PATH_SLIIDERS_TMP))

In [ ]:
this_sliiders = DIR_SCRATCH / "sliiders-fixedadapt.zarr"
this_sliiders_seg = DIR_SCRATCH / "sliiders-fixedadapt-seg.zarr"
output = DIR_SCRATCH / "pyciam-output-fixedadapt.zarr"
tmppath = DIR_SCRATCH / "pyciam-tmp.zarr"

execute_pyciam(
    PATH_PARAMS,
    this_sliiders,
    [PATH_SLR_INT],
    ["local-scc"],
    PATH_REFA,
    econ_input_path_seg=this_sliiders_seg,
    output_path=output,
    tmp_output_path=tmppath,
    no_surge_check=True,
    **common_kwargs,
)

### Run with global average rho